[View in Colaboratory](https://colab.research.google.com/github/juzkev/MBTI-Type/blob/master/MBTI.ipynb)

In [1]:
# Google Colab specific setup for Kaggle API
!pip install kaggle
!mkdir /content/.kaggle
!touch /content/.kaggle/kaggle.json
!echo {\"username\":\"kevinkoh\",\"key\":\"8f58e2b87abd45706b57cd5c80ec917c\"} > /content/.kaggle/kaggle.json
!chmod 600 /content/.kaggle/kaggle.json
!kaggle datasets download -d datasnaek/mbti-type -f mbti_1.csv -p /content
!unzip -o mbti_1.csv.zip -d /input

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
mkdir: cannot create directory ‘/content/.kaggle’: File exists
mbti_1.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  mbti_1.csv.zip
  inflating: /input/mbti_1.csv       


In [0]:
import numpy as np
import pandas as pd
import datetime

In [0]:
my_df = pd.read_csv('/input/mbti_1.csv')

In [73]:
my_df.head()


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [51]:
for i in range (0,5):
  print(my_df.posts[i])

'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...|

In [76]:
# Consider spliting personality type along E-I, S-N, T-F, J-P axis
my_df2 = (my_df.assign(EI = lambda df: df.type.str.extract('([EI])', expand=False))
               .assign(SN = lambda df: df.type.str.extract('([SN])', expand=False))
               .assign(TF = lambda df: df.type.str.extract('([TF])', expand=False))
               .assign(JP = lambda df: df.type.str.extract('([JP])', expand=False))
         )
my_df2.head()
# For use in modelling in later stages

,type,posts,EI,SN,TF,JP
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,I,N,F,J
1,ENTP,'I'm finding the lack of me in these posts ver...,E,N,T,P
2,INTP,'Good one _____ https://www.youtube.com/wat...,I,N,T,P
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",I,N,T,J
4,ENTJ,'You're fired.|||That's another silly misconce...,E,N,T,J


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!pip install -q torchtext

import torchtext

In [7]:
!pip install spacy
!python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
    100% |████████████████████████████████| 17.6MB 2.0MB/s 
    100% |████████████████████████████████| 92kB 18.7MB/s 
    100% |████████████████████████████████| 1.2MB 16.8MB/s 
    100% |████████████████████████████████| 51kB 7.9MB/s 
    100% |████████████████████████████████| 194kB 20.7MB/s 
    100% |████████████████████████████████| 71kB 21.3MB/s 
    100% |████████████████████████████████| 604kB 15.1MB/s 
    100% |████████████████████████████████| 389kB 18.6MB/s 
    100% |████████████████████████████████| 143kB 22.6MB/s 
    100% |████████████████████████████████| 51kB 14.8MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | /

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for murmurhash ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13
  Running setup.py bdist_wheel for preshed ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/8f/85/06/2d132fb649a6bbcab22487e4147880a55b0dd0f4b18fdfd6b5
  Running setup.py bdist_wheel for thin

 / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/f8/b1/86/c92e4d36b690208fff8471711b85eaa6bc6d19860a86199a09
  Running setup.py bdist_wheel for msgpack-python ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
  Running setup.py bdist_wheel for toolz ... - \ done
  Stored in directory: /content/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built spacy murmurhash cymem preshed thinc pathlib ujson dill regex wrapt cytoolz msgpack-python toolz
Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
    100% |████████████████████████████████| 37.4MB 49.5MB/s 
  Running setup.py install for en-core-web-sm ... - \ done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en

In [0]:
import re
import spacy 
spacy_en = spacy.load('en')
def tokenizer(text): # create a tokenizer function
    text = re.sub(r'\|\|\|', '\n', text) # replace '|||' post spliters with line break
    # To remove stopwords of personality types themselves
    return [tok.text for tok in spacy_en.tokenizer(text)]

TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [58]:
starttime = datetime.datetime.now()
train, val, test = torchtext.data.TabularDataset(
        path='/input/mbti_1.csv', format='csv',
        fields=[('type', LABEL), ('posts', TEXT)]).split([.6, .2, .2])
print(datetime.datetime.now()-starttime)

0:04:46.753087


In [0]:
TEXT.build_vocab(train, vectors="glove.6B.100d")

In [78]:
a = train.examples[0]
a.posts[:20]

["'",
 'wow',
 ',',
 'you',
 'guys',
 'are',
 'awesome',
 '.',
 ' ',
 'that',
 "'s",
 'a',
 'great',
 'starting',
 'point',
 'for',
 'me',
 '.',
 ' ',
 'thanks']